In [22]:
#import necessary packages
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

In [23]:
#creating the soup object
page = requests.get('https://byucougars.com/components/nav/nav.html')
soup = BeautifulSoup(page.text,"html.parser")
results = soup.find(id = "myNavmenu")

In [24]:
#finding men's sports and links
menresults = results.select('li[class = "col-md-5 col-sm-5 mens-sports"]')
mentitle = menresults[0].select('li[class*="sports-menu-sport"] > a')
msid = []
for i in range(len(mentitle)):
    att = mentitle[i].attrs
    values = att['ui-sref']
    sportsid = re.search("'(.*?)'", values)
    msid.append(sportsid.group(1))
mensportslist = [i.text for i in mentitle]
print(mensportslist)
print(msid)

['Baseball', 'Basketball', 'Cross Country', 'Football', 'Golf', 'Swimming & Diving', 'Tennis', 'Track & Field', 'Volleyball']
['baseball', 'm-basketball', 'm-cross-country', 'football', 'm-golf', 'm-swimming-diving', 'm-tennis', 'm-track-field', 'm-volleyball']


In [25]:
#finding women's sports and links
womenresults = results.select('li[class*="womens-sport"]')
womentitle = womenresults[0].select('li[class*="sports-menu-sport"] > a')
wsid = []
for i in range(len(womentitle)):
    att = womentitle[i].attrs
    values = att['ui-sref']
    sportsid = re.search("'(.*?)'", values)
    wsid.append(sportsid.group(1))
womensportslist = [i.text for i in womentitle]
print(womensportslist)
print(wsid)

['Basketball', 'Cross Country', 'Golf', 'Gymnastics', 'Soccer', 'Softball', 'Swimming & Diving', 'Tennis', 'Track & Field', 'Volleyball']
['w-basketball', 'w-cross-country', 'w-golf', 'w-gymnastics', 'w-soccer', 'softball', 'w-swimming-diving', 'w-tennis', 'w-track-field', 'w-volleyball']


In [26]:
#finding contents in womens sports
name = []
urls = []
title = []
phone = []
email = []
sport = []
category = []

for i in range(len(wsid)):
    
    # finding out the tid of the staff
    wsport = womensportslist[i]
    wcategory = "Women" 
    url = requests.get('https://byucougars.com/dl/feeds/menu-'+wsid[i])
    js = url.json()
    results = [];
    searchField = "name";
    searchVal = "Staff";
    for i in range(len(js)):
        if(js[i][searchField] == searchVal):
            results.append(js[i])
    tid = results[0]['tid']
    
    # using the tid to locate the staff details
    results1 = [];
    searchField1 = "parent_tid";
    searchVal1 = tid;
    for i in range(len(js)):
        if(js[i][searchField1] == searchVal1):
            results1.append(js[i])
            
    # finding the contents and storing
    for i in range(len(results1)):
        name.append(results1[i]['name'])
        sport.append(wsport)
        category.append(wcategory)
        urlw = results1[i]['field_nav_url']
        if urlw.startswith('http'):
            urls.append(urlw)
        else:
            urls.append('https://byucougars.com'+urlw)
        s = urlw.split("/")
        profileurl = requests.get('https://byucougars.com/dl/feeds/staff-bio-all/'+s[-2])
        p = profileurl.json()
        title.append(p[0]['field_job_title'])
        phone.append(p[0]['field_office_phone'])
        email.append(p[0]['field_email_address'])
    
    #logging the progress
    print(wsport + " is complete")

Basketball is complete
Cross Country is complete
Golf is complete
Gymnastics is complete
Soccer is complete
Softball is complete
Swimming & Diving is complete
Tennis is complete
Track & Field is complete
Volleyball is complete


In [27]:
# storing the result in a dataframe
data = pd.DataFrame({'name' : name,'urls' : urls, 'title' : title, 'phone' : phone,'email' : email,'sport' : sport, 'category' : category})
data

,name,urls,title,phone,email,sport,category
0,Jeff Judkins,https://byucougars.com/staff/w-basketball/1694...,"Women&#039;s Basketball, Head Coach",(801) 422-1265,jeff_judkins@byu.edu,Basketball,Women
1,Ray Stewart,https://byucougars.com/staff/w-basketball/8284...,Assistant Women&#039;s Basketball Coach,(801) 422-7682,ray_stewart@byu.edu,Basketball,Women
2,Melanie Day,https://byucougars.com/staff/w-basketball/1289...,Women&#039;s Basketball Assistant Coach,(801) 422-8155,melanie_day@byu.edu,Basketball,Women
3,Lee Cummard,https://byucougars.com/staff/w-basketball/1168...,Women&#039;s Basketball Assistant Coach,(801) 422-2074,lcummard@byu.edu,Basketball,Women
4,Kim Gibb,https://byucougars.com/staff/w-basketball/1213...,Director of Basketball Operations,(801) 422-1548,kim_parker@byu.edu,Basketball,Women
5,Fayrine Thomas,https://byucougars.com/staff/w-basketball/1701...,Olympic Sports Office Manager,(801) 422-8035,fayrine_thomas@byu.edu,Basketball,Women
6,Ed Eyestone,https://byucougars.com/staff/m-cross-country/1...,Director of Track and Field/Head Cross Countr...,(801) 422-3329,ed_eyestone@byu.edu,Cross Country,Women
7,Diljeet Taylor,https://byucougars.com/staff/m-cross-country/1...,Associate Director of Cross Country and Track ...,(801) 422-7922,diljeet_taylor@byu.edu,Cross Country,Women
8,Isaac Wood,https://byucougars.com/staff/m-cross-country/1...,Director of Operations,(801) 422-1295,Isaac_Wood@byu.edu,Cross Country,Women
9,Carrie Roberts,https://byucougars.com/staff/w-golf/17113/Carr...,Head Women&#039;s Golf Coach,(801) 422-9787,carrie_roberts@byu.edu,Golf,Women


In [28]:
#finding contents in mens sports
name1 = []
urls1 = []
title1 = []
phone1 = []
email1 = []
sport1 = []
category1 = []

for i in range(len(msid)):
    # finding out the tid of the staff
    msport = mensportslist[i]
    mcategory = "Men" 
    url1 = requests.get('https://byucougars.com/dl/feeds/menu-'+msid[i])
    js1 = url1.json()
    results1 = [];
    searchField2 = "name";
    searchVal2 = "Staff";
    for i in range(len(js1)):
        if(js1[i][searchField2] == searchVal2):
            results1.append(js1[i])
    tid1 = results1[0]['tid']
    
    # using the tid to locate the staff details
    results2 = [];
    searchField3 = "parent_tid";
    searchVal3 = tid1;
    for i in range(len(js1)):
        if(js1[i][searchField3] == searchVal3):
            results2.append(js1[i])
            
    # finding the contents and storing
    for i in range(len(results2)):
        sport1.append(msport)
        category1.append(mcategory)
        name1.append(results2[i]['name'])
        url1 = results2[i]['field_nav_url']
        if url1.startswith('http'):
            urls1.append(url1)
        else:
            urls1.append('https://byucougars.com'+url1)
        s1 = url1.split("/")
        profileurl1 = requests.get('https://byucougars.com/dl/feeds/staff-bio-all/'+s1[-2])
        p1 = profileurl1.json()
        try:
            title1.append(p1[0]['field_job_title'])
            phone1.append(p1[0]['field_office_phone'])
            email1.append(p1[0]['field_email_address'])
        except IndexError:
            title1.append("")
            phone1.append("")
            email1.append("")
                
    #logging the progress
    print(msport + " is complete")

Baseball is complete
Basketball is complete
Cross Country is complete
Football is complete
Golf is complete
Swimming & Diving is complete
Tennis is complete
Track & Field is complete
Volleyball is complete


In [29]:
# storing the result in a dataframe
data1 = pd.DataFrame({'name' : name1 ,'urls' : urls1, 'title' : title1, 'phone' : phone1,'email' : email1,'sport' : sport1, 'category' : category1})
data1

,name,urls,title,phone,email,sport,category
0,Mike Littlewood,https://byucougars.com/staff/baseball/94946/Mi...,Baseball Head Coach,(801) 422-5049,mlittlewd@gmail.com,Baseball,Men
1,Trent Pratt,https://byucougars.com/staff/baseball/97058/Tr...,Baseball Associate Head Coach,(801) 422-5064,trent_pratt@byu.edu,Baseball,Men
2,Brent Haring,https://byucougars.com/staff/baseball/97059/Br...,Baseball Assistant Coach/Recruiting Coordinator,(801) 422-5048,brent_haring@byu.edu,Baseball,Men
3,Michael Bradshaw,https://byucougars.com/staff/baseball/1285550/...,Pitching Coach,,michael_bradshaw@byu.edu,Baseball,Men
4,Tuckett Slade,https://byucougars.com/staff/baseball/116918/T...,Director of Baseball Operations,(801) 422-2244,tuckett_slade@byu.edu,Baseball,Men
5,Fayrine Thomas,https://byucougars.com/staff/baseball/17014/Fa...,Olympic Sports Office Manager,(801) 422-8035,fayrine_thomas@byu.edu,Baseball,Men
6,Tom Golightly,https://byucougars.com/staff/baseball/107744/T...,Psychologist,(801) 422-3035,tom_golightly@byu.edu,Baseball,Men
7,Mark Pope,https://byucougars.com/staff/m-basketball/8093...,Head Men&#039;s Basketball Coach,(801) 422-3612,natalie_lainhart@byu.edu,Basketball,Men
8,Chris Burgess,https://byucougars.com/staff/m-basketball/1289...,Men&#039;s Basketball Assistant Coach,(801) 422-5040,chris_burgess@byu.edu,Basketball,Men
9,Cody Fueger,https://byucougars.com/staff/m-basketball/1017...,Men&#039;s Basketball Assistant Coach,(801) 422-6190,cody_fueger@byu.edu,Basketball,Men


In [30]:
#merging both results
alldata = data.append(data1)
alldata

,name,urls,title,phone,email,sport,category
0,Jeff Judkins,https://byucougars.com/staff/w-basketball/1694...,"Women&#039;s Basketball, Head Coach",(801) 422-1265,jeff_judkins@byu.edu,Basketball,Women
1,Ray Stewart,https://byucougars.com/staff/w-basketball/8284...,Assistant Women&#039;s Basketball Coach,(801) 422-7682,ray_stewart@byu.edu,Basketball,Women
2,Melanie Day,https://byucougars.com/staff/w-basketball/1289...,Women&#039;s Basketball Assistant Coach,(801) 422-8155,melanie_day@byu.edu,Basketball,Women
3,Lee Cummard,https://byucougars.com/staff/w-basketball/1168...,Women&#039;s Basketball Assistant Coach,(801) 422-2074,lcummard@byu.edu,Basketball,Women
4,Kim Gibb,https://byucougars.com/staff/w-basketball/1213...,Director of Basketball Operations,(801) 422-1548,kim_parker@byu.edu,Basketball,Women
...,...,...,...,...,...,...,...
49,Rick McWhorter,https://byucougars.com/staff/w-track-field/129...,,1 (479) 200-9499,rick.mcwhorter@gmail.com,Track & Field,Men
50,Isaac Wood,https://byucougars.com/staff/m-track-field/117...,Director of Operations,(801) 422-1295,Isaac_Wood@byu.edu,Track & Field,Men
51,Shawn Olmstead,https://byucougars.com/staff/m-volleyball/1707...,Men&#039;s Volleyball Head Coach,(801) 422-6473,shawn_olmstead@byu.edu,Volleyball,Men
52,Devin Young,https://byucougars.com/staff/m-volleyball/1286...,Assistant Coach,(801) 422-8291,devin_young@byu.edu,Volleyball,Men


In [31]:
#storing in csv
alldata.to_csv('byusports.csv')